# Age word problem Finder

## TFIDF

In [2]:
# Find age word problems using tf_idf cos similarity measure
import re
import json
import os
from nltk.tokenize import TreebankWordTokenizer
from nltk.stem import WordNetLemmatizer
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity as cosSim
import pickle

In [3]:
#LDA(Linear Discriminant Analysis) 
#Using LDA scikit learn
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import cross_val_score
lda = LDA()
dir = ".\Dataset"
docs = []
y=[]
class_1_Text = []
class_0_Text = []
doc_index = []
count = 0
for file in os.listdir(dir):
    path = dir+"\\"
    file_name = path+file
    cl = -1
    if(re.match("ageText[.]txt", file)):
        print("match:"+file)
        fr = open(file_name, "r", encoding='cp1252')
        cl = 1  # Age
    elif(re.match("common.*[.]txt", file) or re.match("illin.*[.]txt", file)):
        fr = open(file_name, "r", encoding='cp1252')
        cl = 0  # Non Age

    if cl==1:
        for line in fr:
            class_1_Text.append(line)
            doc_index.append("Agedoc"+str(count))
            count = count+1
            # print(temp)
    elif cl==0:
        for line in fr:
            class_0_Text.append(line)
            doc_index.append("Agedoc"+str(count))
            count = count + 1

# print(np.sum(y),np.sum(1-np.array(y)))
print(np.shape(doc_index))
# corpus = [doc for doc,mask in zip(docs,y) if mask]
age_docs = class_1_Text
nonage_docs = [class_0_Text[np.random.randint(0,len(class_1_Text))] for i in range(0,len(class_1_Text))]
# print(nonage_docs)
corpus = age_docs + nonage_docs
vectorizer = TfidfVectorizer(input='content',min_df=1, encoding='cp1252')
model = vectorizer.fit_transform(corpus)
with open('tfidf_model.pk','wb') as f:
    pickle.dump(vectorizer,f) 
model = np.array(model.toarray())
y = np.append(np.ones(len(age_docs)),np.zeros(len(nonage_docs)))
doc_index = ['agedoc'+str(i) if val==1 else '!agedoc'+str(i) for i,val in enumerate(y)]
lda = LDA(n_components=1).fit(model,y)
prob_list=[]
text_list=[]
filename_list = []
sim_age_list = []
for file in os.listdir(os.path.join(dir, "TestData")):
    fr = open(os.path.join(dir, "TestData", file), encoding="cp1252")
    for line in fr:
        tfidf_vec = vectorizer.transform([line])
        predict = lda.predict(tfidf_vec)
        prob = lda.predict_proba(tfidf_vec).flatten()
        if(prob[1]>.9):
            prob_list.append(prob[1])
            text_list.append(line)
            filename_list.append(file)
            print(line)
tfidf_df = pd.DataFrame(zip(prob_list,text_list, filename_list), columns='tfidf_prob_Score Text Doc_Name'.split(
)).sort_values(by=['tfidf_prob_Score'], ascending=False)
tfidf_df.to_csv("LDATest.csv")
# display(Ldiadf)
# print(list(zip(lda.predict(ldia_doc_topic_vec),y)))

lda =LDA()
scores = cross_val_score(lda,model,y,cv=5)
print(scores.mean(),scores.std())


match:ageText.txt
(1286,)
A store is selling compact discs for 10.50 dollars and 8.50 dollars. You buy 10 discs and spend a total of 93 dollars. How many compact discs did you buy that cost 10.50 dollars? How many did you buy that cost 8.50 dollars? 

Bob invested 6100 dollars in 2 funds , which pay 7 % and 6 % simple interest respectively. The combined interest he earned for both funds was 405 dollars for one year. How many dollars was invested at 7 %? How much was invested at 6 %? 

Your teacher is giving you a test worth 100 points and containing 40 questions. There are 2 point and 4 point questions on the test. How many two point questions are there? How many 4 point questions? 

You have 160 dollars and save 7 dollars per week. Your friend has 210 dollars and saves 5 dollars per week. After how many weeks will each of you have saved the same amount of money?

A 20-foot board is cut into 2 pieces. Twice the length of the shorter piece is 4 feet more than the length of the longer pi

In [ ]:
#Find centroid
# np.shape(y),np.shape(model)
from sklearn.preprocessing import MinMaxScaler
from pugnlp.stats import confusion 
age_doc_count = np.sum(y)
nage_doc_count = np.sum(1-y)
np.shape(model.T),np.shape(y.T)
            
age_centroid = np.matmul(model.T, y.T)/age_doc_count
nage_centroid = np.matmul(model.T, (1-y).T)/nage_doc_count
LDA_model = age_centroid - nage_centroid
age_scores = np.dot(model,LDA_model)
age_scores = MinMaxScaler().fit_transform(age_scores.reshape(-1,1))
age_predict = (age_scores>.5).astype(int)
accuracy = (1-np.mean(np.abs(age_predict-y)))
print(accuracy)
# print((age_predict,y))
dataF = pd.DataFrame({'prediction': age_predict.flat, 'label': y})
print(confusion(dataF['prediction label'.split()]))

dataF[dataF['prediction'] != dataF['label']]
# print(age_scores.round(2))
# f = open(path+"alg514questions.txt", "r")
scores=[]
text =[]
file_name = []
for file in os.listdir(os.path.join(dir, "TestData")):
    fr = open(os.path.join(dir, "TestData", file), encoding="cp1252")
    for line in fr:
        transformed_line = vectorizer.transform([line])
        # print(transformed_line.toarray())
        age_score = np.dot(transformed_line.toarray().flat, LDA_model)
        scores.append(age_score)
        text.append(line)
        file_name.append(file)
scores = MinMaxScaler().fit_transform(np.array(scores).reshape(-1,1))
labels = pd.DataFrame({'label':y})
resultDf = pd.DataFrame({"Text": text, "Score": scores.flat,"File Name":file_name})
display(resultDf[resultDf['Score'] > .5].sort_values(by="Score",ascending=False).to_csv("resultLDA.csv"))


## LSA

In [ ]:
# Using LSA (Latent Semantic Analysis)
    # U,s,vt = np.linalg.svd(model.T) #Using numpy
    # LSAdf = pd.DataFrame(U, index=vectorizer.vocabulary_.keys()
    # )
    # LSAdf
    # s.round(1)
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity as cosSim
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
import seaborn
pd.options.display.width = 140
dir = ".\Dataset"
docs = []
y=[]
class_1_Text = []
class_0_Text = []
for file in os.listdir(dir):
    path = dir+"\\"
    file_name = path+file
    cl = -1
    if(re.match("ageText[.]txt", file)):
        print("match:"+file)
        fr = open(file_name, "r", encoding='cp1252')
        cl = 1  # Age
        
    elif(re.match("common.*[.]txt", file) or re.match("illin.*[.]txt", file)):
        fr = open(file_name, "r", encoding='cp1252')
        cl = 0  # Non Age

    if cl==1:
        for line in fr:
            class_1_Text.append(line)
            # print(temp)
    elif cl==0:
        for line in fr:
            class_0_Text.append(line)
# print(np.sum(y),np.sum(1-np.array(y)))

# corpus = [doc for doc,mask in zip(docs,y) if mask]
age_docs = class_1_Text
nonage_docs = [class_0_Text[np.random.randint(0,len(class_1_Text))] for i in range(0,len(class_1_Text))]
# print(nonage_docs)
corpus = age_docs + nonage_docs
y = np.append(np.ones(len(age_docs)),np.zeros(len(nonage_docs)))
# print(y)
# print(len(corpus))
model = TfidfVectorizer(min_df=1, encoding='cp1252')
tfidf = model.fit_transform(corpus)
tfidf = np.array(tfidf.toarray())

# len(docs)

pca = PCA(n_components=min(np.shape(tfidf))-100)
# tfidf = tfidf - tfidf.mean()
result_pca = pca.fit_transform(tfidf)
column,ind = (zip(*sorted(model.vocabulary_.items(),key = lambda x:x[1])))
resultDF_pca = pd.DataFrame(pca.components_,columns=column)
# resultDF_pca.to_csv("pcaLSA.csv")
age_pca  = [val for val,mask in zip(result_pca,y) if mask]
nonage_pca = [val for val,mask in zip(result_pca,y) if 1-mask]
total_pca  = np.vstack((age_pca,nonage_pca))
# total_pca = age_pca
# print(np.shape(total_pca))
labels  = ['doc'+str(i) for i in range(len(age_pca))]+['doc'+str(i)+"!" for i in range(len(age_pca),len(age_pca)+len(nonage_pca))]
sim = cosSim(total_pca,total_pca)
# print(np.shape(sim))
# !!!!! simDf = pd.DataFrame(sim,index=labels, columns=labels) # Important result from LSA
# display(simDf)
# resultDF_pca[1][resultDF_pca[1]>resultDF_pca[0]].sort_values(ascending=False)[:20]
# resultDF_pca.plot(kind='scatter',x='x',y='y')
# resultDF_pca[resultDF_pca['y']>1]
# pca.components_
# print(np.shape(result_pca))
# (pca.explained_variance_ratio_.round(2))
# # resultDF_pca['is more than thrice as old as his son'.split()].T.sum()
# centroid_age_pca = np.mean(age_pca,axis=0)
# centroid_non_age_pca = np.mean(nonage_pca, axis=0)
print(dir)
sim_age_list=[]
text_list=[]
filename_list = []
lda_model = LDA(n_components=1).fit(total_pca,y)

for file in os.listdir(os.path.join(dir,"TestData")):
    fr = open(os.path.join(dir,"TestData",file), encoding="cp1252")
    for line in fr:
        tfidf_vec = model.transform([line])
        topic_vec = pca.transform(tfidf_vec.toarray())
        # sim_vec = cosSim(age_pca, topic_vec.reshape(1, -1))
        predict = lda_model.predict(topic_vec)
        prob = lda_model.predict_proba(topic_vec)
        # print(prob)
        # print(predict.flatten())
        # sim_age = cosSim(centroid_age_pca.reshape(1,-1), topic_vec.reshape(1,-1))
        
        # print(sim_vec,sim_age)
        # break
        # sim_nonage = cosSim(centroid_non_age_pca.reshape(1, -1),topic_vec.reshape(1, -1))
        # print(sim_age,sim_nonage)
        if(predict and prob.flatten()[1] > .9):
            sim_age_list.append(prob.flatten()[1])
            text_list.append(line)
            filename_list.append(file)

LSAdf = pd.DataFrame(zip(sim_age_list,text_list,filename_list), columns='LSA_Score Text Doc_Name'.split()).sort_values(by='LSA_Score',ascending = False)
LSAdf.to_csv("LSATestData.csv")
display(LSAdf)
# cosSim()


## LDiA

In [ ]:
# LDIA
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation as LDiA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import cross_val_score
# from sklearn.
import re
counter = CountVectorizer()
dir = ".\Dataset"
docs = []
y=[]
class_1_Text = []
class_0_Text = []
doc_index = []
count = 0
for file in os.listdir(dir):
    path = dir+"\\"
    file_name = path+file
    cl = -1
    if(re.match("ageText[.]txt", file)):
        print("match:"+file)
        fr = open(file_name, "r", encoding='cp1252')
        cl = 1  # Age
    elif(re.match("common.*[.]txt", file) or re.match("illin.*[.]txt", file)):
        fr = open(file_name, "r", encoding='cp1252')
        cl = 0  # Non Age

    if cl==1:
        for line in fr:
            class_1_Text.append(line)
            doc_index.append("Agedoc"+str(count))
            count = count+1
            # print(temp)
    elif cl==0:
        for line in fr:
            class_0_Text.append(line)
            doc_index.append("Agedoc"+str(count))
            count = count + 1

# print(np.sum(y),np.sum(1-np.array(y)))
print(np.shape(doc_index))
# corpus = [doc for doc,mask in zip(docs,y) if mask]
age_docs = class_1_Text
nonage_docs = [class_0_Text[np.random.randint(0,len(class_1_Text))] for i in range(0,len(class_1_Text))]
# print(nonage_docs)
corpus = age_docs + nonage_docs
y = np.append(np.ones(len(age_docs)),np.zeros(len(nonage_docs)))
doc_index = ['agedoc'+str(i) if val==1 else '!agedoc'+str(i) for i,val in enumerate(y)]
# print(np.shape(doc_index))

bow = pd.DataFrame(counter.fit_transform(corpus).toarray(),index=doc_index)
terms,index = zip(*sorted(counter.vocabulary_.items(), key = lambda x:x[1]))
bow.columns = terms
# ldia_model = 
ldia = LDiA(n_components=min(np.shape(tfidf)) -
            100, learning_method="batch").fit(bow)
ldia_components =pd.DataFrame(ldia.components_.T,index=terms)
ldia_doc_topic_vec = ldia.transform(bow)
ldia_doc_topic_vec_df  = pd.DataFrame(ldia_doc_topic_vec,index=doc_index)
age_centroid = np.mean(np.array([doc for doc,mask in zip(ldia_doc_topic_vec,y) if mask]),axis=0)
# print(np.array([doc for doc, mask in zip(ldia_doc_topic_vec, y) if mask]))
non_age_centroid = np.mean(np.array([doc for doc, mask in zip(ldia_doc_topic_vec, y) if 1-mask]))
# print(age_centroid)
# display(ldia_components[0].sort_values(ascending=False)[:20])
# display(ldia_doc_topic_vec_df)
lda = LDA(n_components=1).fit(ldia_doc_topic_vec,y)
prob_list=[]
text_list=[]
filename_list = []
sim_age_list = []
for file in os.listdir(os.path.join(dir, "TestData")):
    fr = open(os.path.join(dir, "TestData", file), encoding="cp1252")
    for line in fr:
        ldia_bow_vec = counter.transform([line])
        # print(np.shape(ldia_bow_vec))
        ldia_topic_vec = ldia.transform(ldia_bow_vec)
        # print(ldia_topic_vec)
        predict = lda.predict(ldia_topic_vec)
        prob = lda.predict_proba(ldia_topic_vec).flatten()
        # print(predict.flatten())
        sim_age = cosSim(age_centroid.reshape(1,-1), ldia_topic_vec.reshape(1,-1)).flatten()
        # print(sim_age,predict,prob[1])
        # print(sim_vec,sim_age)
        # break
        # sim_nonage = cosSim(centroid_non_age_pca.reshape(1, -1),topic_vec.reshape(1, -1))
        # print(sim_age,sim_nonage)
        if(prob[1]>.5 and sim_age>.7):
            prob_list.append(prob[1])
            sim_age_list.append(sim_age[0])
            text_list.append(line)
            filename_list.append(file)
            print(line)
Ldiadf = pd.DataFrame(zip(prob_list,sim_age_list, text_list, filename_list),columns='Ldia_prob_Score Ldia_sim_score Text Doc_Name'.split()).sort_values(by=['Ldia_prob_Score','Ldia_sim_score'], ascending=False)
Ldiadf.to_csv("LDiATestData.csv")
# display(Ldiadf)
# print(list(zip(lda.predict(ldia_doc_topic_vec),y)))
print(1 - np.mean(np.abs(lda.predict(ldia_doc_topic_vec)-y)))
lda =LDA()
scores = cross_val_score(lda,ldia_doc_topic_vec,y,cv=5)
print(scores.mean(),scores.std())

In [ ]:
list(zip(sim_age_list,text_ealist,filename_list))
[1,2,3]+[4,5,6]
np.zeros(5)
np.append([1,2,3],[4,5])

## Word2Vec
Similarity using word2vec and doc2vec


In [ ]:
from gensim.models.keyedvectors import KeyedVectors
gWord2vec_path = "D:\Research\OntologyAge\Code\GoogleNews-vectors-negative300.bin.gz"
word_vec = KeyedVectors.load_word2vec_format(gWord2vec_path,binary=True)



In [ ]:
word_vec.most_similar(positive=["age","old","young","year"])
word_vec.similarity('young','age')

In [ ]:
import nltk.data
from nltk.tokenize import wordpunct_tokenize
from gensim.models.word2vec import Word2Vec
from gensim.models import KeyedVectors
import numpy as np
from sklearn.decomposition import PCA
from matplotlib import pyplot
import glob
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
data = []
# data=open(".\Dataset\AgeText.txt").read()
for file in glob.glob(".\Dataset\*.txt"):
    fp=open(file,encoding='utf-8', errors='ignore')
    data.append(fp.read())
data = "".join(data)
print(len(data))
word2vec_token = [wordpunct_tokenize(sent) for sent in tokenizer.tokenize(data)]
num_feature = 300
min_word_count = 2
num_worker = 6
window_sz  = 8
subsampling = 1e-3
model = Word2Vec(word2vec_token,workers=num_worker,vector_size= num_feature,min_count=min_word_count,window=window_sz,sample=subsampling)
# model.init_sims(replace=True)
model.save("ageWord2VecModel")
# wv = KeyedVectors.load("ageWord2VecModel",mmap='r')
model.wv.most_similar('age', topn=20)
# len(model.wv.key_to_index)
sorted([i for i in model.wv.key_to_index.items()],key = lambda x: x[1])
X = model.wv[model.wv.index_to_key]
pca = PCA(n_components=2)
# age_2d = pca.fit_transform(model.wv)
result = pca.fit_transform(X)
model.wv.key_to_index.keys()
np.shape(result)
pyplot.scatter(result[:,0],result[:,1])
for i,word in enumerate(model.wv.index_to_key):
    pyplot.annotate(word,xy=(result[i,0],result[i,1]))
pyplot.plot()
# len(data)

In [ ]:
import multiprocessing
import nltk
num_cores = multiprocessing.cpu_count()
from gensim.models.doc2vec import TaggedDocument, Doc2Vec
from gensim.utils import simple_preprocess
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
corpus = []
for line in open("./Dataset/ageText.txt"):
    corpus.append(line)
tag_corpus = []
for i,doc in enumerate(corpus):
    tag_doc = TaggedDocument(simple_preprocess(doc),[i])
    tag_corpus.append(tag_doc)
model = Doc2Vec(vector_size = 500,workers = num_cores,epochs=1000)
model.build_vocab(tag_corpus)
model.train(tag_corpus,total_examples=model.corpus_count,epochs=model.epochs)
doc2vec_model = model


In [ ]:
import numpy as np
docvecs=[] 
for i,doc in enumerate(corpus):
    docvecs.append(doc2vec_model.dv[i])
doc_vec_centroid = np.mean(np.array(docvecs),axis=0,keepdims=True)
np.shape(doc_vec_centroid)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity as cosSim
import pandas as pd
dir = '.\Dataset'
sim_age_list=[]
text_list = []
filename_list= []
for file in os.listdir(os.path.join(dir, "TestData")):
    fr = open(os.path.join(dir, "TestData", file), encoding="cp1252")
    for line in fr:
        doc_vec = doc2vec_model.infer_vector(simple_preprocess(line),epochs=100)
        
        sim_age = cosSim(doc_vec.reshape(1,-1),doc_vec_centroid)
        if(sim_age > .7):
            sim_age_list.append(sim_age[0])
            text_list.append(line)
            filename_list.append(file)
            print(line)
doc2vec_DF = pd.DataFrame(zip(sim_age_list, text_list, filename_list), columns='doc2vec_sim_score Text Doc_Name'.split(
)).sort_values(by=['doc2vec_sim_score'], ascending=False)
doc2vec_DF.to_csv("doc2vecTestData.csv")
# display(Ldiadf)
# print(list(zip(lda.predict(ldia_doc_topic_vec),y)))
# print(1 - np.mean(np.abs(lda.predict(ldia_doc_topic_vec)-y)))
# lda = LDA()
# scores = cross_val_score(lda, ldia_doc_topic_vec, y, cv=5)
# print(scores.mean(), scores.std())

# doc2vec_model.infer_vector(simp)


## Convolution Neural Nets (CNN)

In [6]:
import glob
import os
from random import shuffle
def preprocess_data(path):
    pos_path = os.path.join(path,'pos')
    neg_path = os.path.join(path,'neg')
    pos_label = 1
    neg_label = 0
    dataset = []
    print(pos_path)
    for filename in glob.glob(os.path.join(pos_path,'*.txt')):
        with open(filename,'r') as f:
            dataset.append((pos_label,f.read()))
    for filename in glob.glob(os.path.join(neg_path,'*.txt')):
        with open(filename,'r') as f:
            dataset.append((neg_label,f.read()))
    shuffle(dataset)

    return dataset

In [7]:
from nltk.tokenize import TreebankWordTokenizer
import numpy as np
from gensim.models.keyedvectors import KeyedVectors
word_vectors =KeyedVectors.load("ageWord2VecModel").wv
# with open("tfidf_model.pk",'rb') as f:
#     vectorizer = pickle.load(f)
print(np.shape(word_vectors[0]))
# print(vectorizer.vocabulary_)
def tokenize_vectorize(dataset):
    tokenizer = TreebankWordTokenizer()
    vectorized_data = []
    expected = []
    for sample in dataset:
        # print(sample)
        tokens = tokenizer.tokenize(sample[1])
        expected.append(sample[0])
        vecs =[]
        for token in tokens:
            try:
                vecs.append(word_vectors[token])
            except KeyError:
                pass
        # expected.append(sample[0])
        vectorized_data.append(vecs)
        # vectorized_data.append(vectorizer.transform([sample[1]]).toarray().reshape(-1,1))
    return (vectorized_data,expected)   


(300,)


In [ ]:
import numpy as np
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Conv1D, GlobalMaxPooling1D,MaxPooling1D
from gensim.models.keyedvectors import KeyedVectors
from sklearn.model_selection import train_test_split
dataset = preprocess_data(r'Dataset\input_dataset')
vectorized_data, expected = tokenize_vectorize(dataset)
print(np.shape(vectorized_data[0])[0])
# print(vectorized_data)
maxlen = max([len(i) for i in vectorized_data])
# vectorized_data
emb_dim = len(vectorized_data[0][0])
pad_dataset = [smp[:maxlen] + [[0.]* emb_dim] * (maxlen - len(smp)) for smp in vectorized_data]
x_train, x_test, y_train, y_test = train_test_split(
    pad_dataset, expected, train_size=.8, random_state=42)
# x_train, x_test, y_train, y_test = train_test_split(
#     vectorized_data, expected, train_size=.8, random_state=42)
# print(x_train)
x_train = np.array(x_train)
# y_train = np.reshape(np.array(y_train), (-1, 1))
y_train = np.array(y_train)
x_test = np.array(x_test)
# y_test = np.reshape(np.array(y_test), (-1, 1))
y_test = np.array(y_test)
np.shape(x_train),np.shape(y_train),np.shape(x_test),np.shape(y_test),np.shape(expected)


In [ ]:
filters = 250
kernel_size = 4
hidden_dims = 500
epochs = 150
model = Sequential()
model.add(Conv1D(filters,kernel_size,padding='valid',activation='relu', strides =1, input_shape = np.shape(x_train[0])))
model.add(MaxPooling1D(2))
model.add(Conv1D(filters, kernel_size, padding='valid',
                 activation='relu', strides=1))
model.add(GlobalMaxPooling1D())
model.add(Dense(hidden_dims))
model.add(Dropout(0.2))
model.add(Activation('relu'))
model.add(Dense(1))
model.add(Activation('sigmoid'))
model.summary()
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
model.fit(x_train,y_train,epochs=epochs,validation_data=(x_test,y_test))
model_structure = model.to_json()
with open("cnn_model.json", "w") as json_file:
    json_file.write(model_structure)
model.save_weights("cnn_weights.h5")


In [ ]:
from keras.models import model_from_json
from tqdm import tqdm
with open("cnn_model.json","r") as json_file:
    json_string = json_file.read()
model = model_from_json(json_string)
model.load_weights("cnn_weights.h5")

from sklearn.metrics.pairwise import cosine_similarity as cosSim
import pandas as pd
dir = '.\Dataset'
sim_age_list = []
text_list = []
filename_list = []
for file in tqdm(os.listdir(os.path.join(dir, "TestData"))):
    fr = open(os.path.join(dir, "TestData", file), encoding="cp1252")
    for line in tqdm(fr):
        vec_list, exp = tokenize_vectorize([(-1, line)])
        pad_vec = [smp[:maxlen] + [[0.] * emb_dim] *(maxlen - len(smp)) for smp in vec_list]
        # np.shape(pad_vec)
        sim_age = model.predict(np.array(pad_vec))
        # doc_vec = doc2vec_model.infer_vector(
        #     simple_preprocess(line), epochs=100)

        # sim_age = cosSim(doc_vec.reshape(1, -1), doc_vec_centroid)
        if(sim_age > .7):
            sim_age_list.append(sim_age)
            text_list.append(line)
            filename_list.append(file)
            # print(line)
CNN_df = pd.DataFrame(zip(sim_age_list, text_list, filename_list), columns='cnn_sim_score Text Doc_Name'.split(
)).sort_values(by=['cnn_sim_score'], ascending=False)
CNN_df.to_csv("cnnTestData.csv")


## RNN and LSTM

In [1]:
import glob
import os
from random import shuffle
def preprocess_data(path):
    pos_path = os.path.join(path,'pos')
    neg_path = os.path.join(path,'neg')
    pos_label = 1
    neg_label = 0
    dataset = []
    print(pos_path)
    for filename in glob.glob(os.path.join(pos_path,'*.txt')):
        with open(filename,'r') as f:
            dataset.append((pos_label,f.read()))
    for filename in glob.glob(os.path.join(neg_path,'*.txt')):
        with open(filename,'r') as f:
            dataset.append((neg_label,f.read()))
    shuffle(dataset)

    return dataset

In [2]:
from nltk.tokenize import TreebankWordTokenizer
import numpy as np
from gensim.models.keyedvectors import KeyedVectors
word_vectors =KeyedVectors.load("ageWord2VecModel").wv
# with open("tfidf_model.pk",'rb') as f:
#     vectorizer = pickle.load(f)
print(np.shape(word_vectors[0]))
# print(vectorizer.vocabulary_)
def tokenize_vectorize(dataset):
    tokenizer = TreebankWordTokenizer()
    vectorized_data = []
    expected = []
    for sample in dataset:
        # print(sample)
        tokens = tokenizer.tokenize(sample[1])
        expected.append(sample[0])
        vecs =[]
        for token in tokens:
            try:
                vecs.append(word_vectors[token])
            except KeyError:
                pass
        # expected.append(sample[0])
        vectorized_data.append(vecs)
        # vectorized_data.append(vectorizer.transform([sample[1]]).toarray().reshape(-1,1))
    return (vectorized_data,expected)   


(300,)


In [4]:
import numpy as np
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Conv1D, GlobalMaxPooling1D, MaxPooling1D
from gensim.models.keyedvectors import KeyedVectors
from sklearn.model_selection import train_test_split
dataset = preprocess_data(r'Dataset\input_dataset')
vectorized_data, expected = tokenize_vectorize(dataset)
print(np.shape(vectorized_data[0])[0])
# print(vectorized_data)
maxlen = max([len(i) for i in vectorized_data])
# vectorized_data
emb_dim = len(vectorized_data[0][0])
pad_dataset = [smp[:maxlen] + [[0.] * emb_dim] *
               (maxlen - len(smp)) for smp in vectorized_data]
x_train, x_test, y_train, y_test = train_test_split(
    pad_dataset, expected, train_size=.8, random_state=42)
# x_train, x_test, y_train, y_test = train_test_split(
#     vectorized_data, expected, train_size=.8, random_state=42)
# print(x_train)
x_train = np.array(x_train)
# y_train = np.reshape(np.array(y_train), (-1, 1))
y_train = np.array(y_train)
x_test = np.array(x_test)
# y_test = np.reshape(np.array(y_test), (-1, 1))
y_test = np.array(y_test)
np.shape(x_train), np.shape(y_train), np.shape(
    x_test), np.shape(y_test), np.shape(expected)


Dataset\input_dataset\pos
22


((198, 46, 300), (198,), (50, 46, 300), (50,), (248,))

In [5]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, SimpleRNN, LSTM
from keras.layers.wrappers import Bidirectional

num_neurons  = 60
model = Sequential()
# model.add(Bidirectional(SimpleRNN(num_neurons,return_sequences=True)))#RNN
model.add(LSTM(num_neurons,return_sequences='True'))
model.add(Dropout(.2))
model.add(Flatten())
model.add(Dense(1,activation='sigmoid'))
model.compile('rmsprop','binary_crossentropy', metrics = ['accuracy'])
model.fit(x_train,y_train,epochs=100,validation_data=(x_test,y_test))
model.summary()
model_structure = model.to_json()
with open("simple_lstm_model1.json","w") as json_file:
    json_file.write(model_structure)
model.save_weights("simplelstm_weights1.h5")

Epoch 1/100
7/7 [==============================] - 0s 71ms/step - loss: 0.2681 - accuracy: 0.8788 - val_loss: 0.0123 - val_accuracy: 1.0000
Epoch 2/100
7/7 [==============================] - 0s 16ms/step - loss: 0.0251 - accuracy: 0.9899 - val_loss: 0.0026 - val_accuracy: 1.0000
Epoch 3/100
7/7 [==============================] - 0s 16ms/step - loss: 0.0026 - accuracy: 1.0000 - val_loss: 0.0011 - val_accuracy: 1.0000
Epoch 4/100
7/7 [==============================] - 0s 16ms/step - loss: 0.0011 - accuracy: 1.0000 - val_loss: 5.9232e-04 - val_accuracy: 1.0000
Epoch 5/100
7/7 [==============================] - 0s 16ms/step - loss: 6.5033e-04 - accuracy: 1.0000 - val_loss: 3.2978e-04 - val_accuracy: 1.0000
Epoch 6/100
7/7 [==============================] - 0s 16ms/step - loss: 3.5829e-04 - accuracy: 1.0000 - val_loss: 1.9360e-04 - val_accuracy: 1.0000
Epoch 7/100
7/7 [==============================] - 0s 16ms/step - loss: 2.2445e-04 - accuracy: 1.0000 - val_loss: 1.1430e-04 - val_accuracy:

In [10]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity as cosSim
from keras.models import model_from_json
from tqdm import tqdm
with open("simple_lstm_model1.json", "r") as json_file:
    json_string = json_file.read()
model = model_from_json(json_string)
model.load_weights("simplelstm_weights1.h5")

dir = '.\Dataset'
sim_age_list = []
text_list = []
filename_list = []
for file in tqdm(os.listdir(os.path.join(dir, "TestData"))):
    fr = open(os.path.join(dir, "TestData", file), encoding="cp1252")
    for line in tqdm(fr):
        vec_list, exp = tokenize_vectorize([(-1, line)])
        pad_vec = [smp[:maxlen] + [[0.] * emb_dim] *
                   (maxlen - len(smp)) for smp in vec_list]
        # np.shape(pad_vec)
        sim_age = model.predict(np.array(pad_vec))
        # doc_vec = doc2vec_model.infer_vector(
        #     simple_preprocess(line), epochs=100)

        # sim_age = cosSim(doc_vec.reshape(1, -1), doc_vec_centroid)
        if(sim_age > .9):
            sim_age_list.append(sim_age)
            text_list.append(line)
            filename_list.append(file)
            # print(line)
lstm_df = pd.DataFrame(zip(sim_age_list, text_list, filename_list), columns='lstm_sim_score Text Doc_Name'.split(
)).sort_values(by=['lstm_sim_score'], ascending=False)
lstm_df.to_csv("lstmTestData.csv")


514it [00:20, 24.66it/s]
1557it [01:16, 20.30it/s]
600it [00:23, 25.28it/s]
562it [00:22, 25.44it/s]
1529it [01:00, 25.16it/s]
100%|██████████| 5/5 [03:24<00:00, 40.83s/it]